In [1]:
import pandas as pd
import json
from natsort import natsorted

In [2]:
genera = pd.read_csv('final_genera_75.tsv', delimiter='\t')
parsed_glottolog = pd.read_csv('languages_with_pages.tsv', sep='\t', index_col=0)

In [3]:
genera = pd.merge(genera, parsed_glottolog)[['glottocode', 'genus', 'name', 'pages', 'macroareas']]

In [4]:
macroareas = {}

In [5]:
for macro in genera.macroareas.unique():
    if pd.isna(macro):
        continue
    df = genera[genera.macroareas == macro]
    n_genera = len(df.genus.unique()) + sum(df.genus == 'isolate') - 1
    macroareas[macro.strip()] = n_genera

In [49]:
sum(macroareas.values())

722

In [6]:
macroareas_percentage = {
    x: y / sum(macroareas.values()) for x, y in macroareas.items()
}

In [43]:
{
    x: round(y * 20) for x, y in macroareas_percentage.items()
}

{'Papunesia': 6,
 'South America': 4,
 'Eurasia': 3,
 'North America': 3,
 'Africa': 4,
 'Australia': 1}

In [7]:
def convert_pages(pages):
    pages = eval(pages)
    return [x for x in pages if not isinstance(x, str)]

In [44]:
n_langs = 30
given_genera = ['Tungusic', 'Saami', 'Northwest Caucasian', 'Slavic', 'Marind-Yaqay']
banned_genera = ['Western Plains Austronesian', 'Central Luzon',
                 'Peul-Serer', 'Northern Pama-Nyungan', 'Awju', 'Atayalic', 'Eleman',
                'Berber', 'Ta-Ne-Omotic']

n_by_macroarea = {
    x: round(y * n_langs) for x, y in macroareas_percentage.items()
}

sample_by_macroarea = {}
added_genera = []

for macro in n_by_macroarea:
    n = n_by_macroarea[macro]
    df = genera[genera.macroareas == '\n' + macro].copy()
    df.pages = df.pages.apply(lambda x: sorted(convert_pages(x), reverse=True))
    sorted_genera = df.sort_values('pages', ascending=False).genus.dropna().unique().tolist()
    needed_genera = []
    for gen in given_genera:
        if gen in sorted_genera:
            sorted_genera.remove(gen)
            needed_genera.append(gen)
            n -= 1
    for gen in added_genera:
        if gen in sorted_genera:
            sorted_genera.remove(gen)
    for gen in banned_genera:
        if gen in sorted_genera:
            sorted_genera.remove(gen)
    
    sorted_genera.remove('Creoles and Pidgins')
    sorted_genera.remove('Sign Languages')

    needed_genera.extend(sorted_genera[:n])
    added_genera.extend(needed_genera)
    
    sample_by_macroarea[macro] = needed_genera

In [45]:
for key in sample_by_macroarea:
    print(key, '\t'.join(sample_by_macroarea[key]), sep='\t')

Papunesia	Marind-Yaqay	Oceanic	North Borneo	Kalam-Kobon	Greater Central Philippine	Palauan	Sama-Bajaw	Northern Luzon
South America	Panoan	Nadahup	Japura-Colombia	isolate	Quechuan	Bolivia-Parana
Eurasia	Tungusic	Saami	Northwest Caucasian	Slavic
North America	Mayan	Eskimo	Athapaskan	Haida	Tlingit
Africa	Gbe	Bantu	Barito	Biu-Mandara	Semitic
Australia	Western Pama-Nyungan	Nyulnyulan
